# スタッキング002

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [7]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [8]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [9]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [11]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [14]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [15]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356250, 1057)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [16]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1057)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,0,0,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0,0,0,0


In [17]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,1,0,0,0,0,0,0,0,0,0
1,307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,0,0,0,0,0,0,0,0
2,307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,1,0,1,0,1,0,1,0
3,307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,1,0,1,0,1,0,1,0
4,307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,1,0,0,0,0,0,0,0,0,0


In [18]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,0,0,0,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0,0,0,0


In [19]:
### level_0を削除
app_train.drop('level_0', axis=1, inplace=True)
app_test.drop('level_0', axis=1, inplace=True)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0,0,0
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,0,0,0,0,0,0,0
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,1,0,1,0,1,0
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,1,0,1,0,1,0
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0,0,0


### modelを生成

In [20]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [21]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [22]:
# infをnanへ置換
app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,1,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,1,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,1,0,0,0,1,0,1,0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,1,0,0,0,0,0,0,0,0


In [23]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0,0,0
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,0,0,0,0,0,0,0
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,1,0,1,0,1,0
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,1,0,1,0,1,0
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [25]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018845,...,1,0,0,0,0,0,0,0,0,0
100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035797,...,0,1,0,0,0,0,0,0,0,0
100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019104,...,0,1,1,0,1,0,1,0,1,0
100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,...,0,1,1,0,1,0,1,0,1,0
100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010033,...,1,0,0,0,0,0,0,0,0,0


In [26]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,-0.232482,-0.146484,...,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0
100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,-0.907160,1.079102,...,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0
100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,0.217303,-0.127563,...,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,1.601419,0.0,1.601509,0.0
100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,2.578675,0.399658,...,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,1.601419,0.0,1.601509,0.0
100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,0.206059,-0.783203,...,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0


In [27]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,-0.232482,...,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0
1,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,-0.907160,...,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0
2,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,0.217303,...,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,1.601419,0.0,1.601509,0.0
3,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,2.578675,...,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,1.601419,0.0,1.601509,0.0
4,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,0.206059,...,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.624446,0.0,-0.624411,0.0


In [28]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100002,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.165405,-0.479980,...,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0
1,100003,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.585938,1.465820,...,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0
2,100004,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.391602,-1.019531,...,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0
3,100006,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.175171,-0.614746,...,-1.056641,1.190430,1.684570,-0.150513,-0.498047,0.0,1.601562,0.0,1.601562,0.0
4,100007,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.358887,-0.075073,...,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0


In [29]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,100001,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075134,-0.447998,-0.232422,...,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0
1,100005,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.666016,-0.907227,...,-1.056641,1.190430,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0
2,100013,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.912109,0.217285,...,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0,1.601562,0.0,1.601562,0.0
3,100028,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.495117,2.578125,...,-1.056641,1.190430,1.684570,-0.150513,2.007812,0.0,1.601562,0.0,1.601562,0.0
4,100038,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.337646,0.206055,...,0.946777,-0.840332,-0.593262,-0.150513,-0.498047,0.0,-0.624512,0.0,-0.624512,0.0


In [30]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [31]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [32]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Lgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[LightGBM] [Info] Number of positive: 18633, number of negative: 211996
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.596214 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167524
[LightGBM] [Info] Number of data points in the train set: 230629, number of used features: 1047
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080792 -> initscore=-2.431633
[LightGBM] [Info] Start training from score -2.431633
[1]	train's auc: 0.69671	valid's auc: 0.687605
[2]	train's auc: 0.725879	valid's auc: 0.716655
[3]	train's auc: 0.739056	valid's auc: 0.730648
[4]	train's auc: 0.741395	valid's auc: 0.731629
[5]	train's auc: 0.745349	valid's auc: 0.737509
[6]	train's auc: 0.746895	valid's auc: 0.73823
[7]	train's auc: 0.747794	valid's auc: 0.739488
[8]	train's auc: 0.746755	valid's auc: 0.7378
[9]	train's auc: 0.748865	valid's auc: 0.739723
[10]	train's auc: 0.750222	valid's auc: 0.741371
[11]	train's auc: 0.

[163]	train's auc: 0.781186	valid's auc: 0.767614
[164]	train's auc: 0.781301	valid's auc: 0.767686
[165]	train's auc: 0.78148	valid's auc: 0.767788
[166]	train's auc: 0.781607	valid's auc: 0.76786
[167]	train's auc: 0.78166	valid's auc: 0.767859
[168]	train's auc: 0.781918	valid's auc: 0.768117
[169]	train's auc: 0.782091	valid's auc: 0.768257
[170]	train's auc: 0.782228	valid's auc: 0.7683
[171]	train's auc: 0.78223	valid's auc: 0.768293
[172]	train's auc: 0.782404	valid's auc: 0.768412
[173]	train's auc: 0.782548	valid's auc: 0.768532
[174]	train's auc: 0.782536	valid's auc: 0.768507
[175]	train's auc: 0.7827	valid's auc: 0.768649
[176]	train's auc: 0.78272	valid's auc: 0.768664
[177]	train's auc: 0.782897	valid's auc: 0.768805
[178]	train's auc: 0.782927	valid's auc: 0.768788
[179]	train's auc: 0.782944	valid's auc: 0.768765
[180]	train's auc: 0.783213	valid's auc: 0.769049
[181]	train's auc: 0.783312	valid's auc: 0.769111
[182]	train's auc: 0.783454	valid's auc: 0.769244
[183]	tra

[333]	train's auc: 0.79648	valid's auc: 0.77766
[334]	train's auc: 0.796544	valid's auc: 0.777697
[335]	train's auc: 0.796604	valid's auc: 0.777719
[336]	train's auc: 0.796661	valid's auc: 0.777751
[337]	train's auc: 0.796727	valid's auc: 0.777776
[338]	train's auc: 0.796832	valid's auc: 0.777829
[339]	train's auc: 0.796921	valid's auc: 0.777871
[340]	train's auc: 0.796987	valid's auc: 0.777903
[341]	train's auc: 0.797078	valid's auc: 0.777956
[342]	train's auc: 0.797136	valid's auc: 0.777964
[343]	train's auc: 0.797261	valid's auc: 0.778075
[344]	train's auc: 0.797341	valid's auc: 0.778102
[345]	train's auc: 0.797411	valid's auc: 0.77814
[346]	train's auc: 0.797489	valid's auc: 0.778177
[347]	train's auc: 0.797558	valid's auc: 0.77821
[348]	train's auc: 0.797609	valid's auc: 0.778252
[349]	train's auc: 0.797669	valid's auc: 0.778286
[350]	train's auc: 0.797723	valid's auc: 0.778306
[351]	train's auc: 0.797784	valid's auc: 0.778333
[352]	train's auc: 0.797869	valid's auc: 0.778367
[353

[500]	train's auc: 0.807343	valid's auc: 0.783179
[501]	train's auc: 0.807413	valid's auc: 0.783229
[502]	train's auc: 0.807472	valid's auc: 0.783254
[503]	train's auc: 0.807536	valid's auc: 0.783267
[504]	train's auc: 0.807609	valid's auc: 0.783297
[505]	train's auc: 0.807679	valid's auc: 0.783327
[506]	train's auc: 0.807746	valid's auc: 0.783346
[507]	train's auc: 0.807801	valid's auc: 0.783366
[508]	train's auc: 0.807845	valid's auc: 0.783391
[509]	train's auc: 0.807901	valid's auc: 0.783399
[510]	train's auc: 0.807971	valid's auc: 0.783449
[511]	train's auc: 0.808044	valid's auc: 0.783493
[512]	train's auc: 0.808077	valid's auc: 0.783492
[513]	train's auc: 0.808176	valid's auc: 0.783555
[514]	train's auc: 0.808233	valid's auc: 0.783571
[515]	train's auc: 0.808315	valid's auc: 0.783636
[516]	train's auc: 0.808364	valid's auc: 0.783656
[517]	train's auc: 0.808423	valid's auc: 0.783656
[518]	train's auc: 0.808473	valid's auc: 0.783672
[519]	train's auc: 0.808535	valid's auc: 0.783706


[666]	train's auc: 0.81638	valid's auc: 0.786806
[667]	train's auc: 0.816411	valid's auc: 0.786804
[668]	train's auc: 0.816454	valid's auc: 0.786816
[669]	train's auc: 0.816498	valid's auc: 0.786838
[670]	train's auc: 0.816539	valid's auc: 0.786853
[671]	train's auc: 0.816585	valid's auc: 0.78688
[672]	train's auc: 0.816635	valid's auc: 0.786889
[673]	train's auc: 0.816684	valid's auc: 0.7869
[674]	train's auc: 0.816742	valid's auc: 0.786911
[675]	train's auc: 0.816785	valid's auc: 0.786918
[676]	train's auc: 0.816837	valid's auc: 0.786949
[677]	train's auc: 0.816912	valid's auc: 0.786997
[678]	train's auc: 0.81697	valid's auc: 0.787024
[679]	train's auc: 0.817006	valid's auc: 0.787028
[680]	train's auc: 0.817052	valid's auc: 0.787031
[681]	train's auc: 0.817114	valid's auc: 0.787037
[682]	train's auc: 0.817161	valid's auc: 0.787057
[683]	train's auc: 0.817213	valid's auc: 0.787064
[684]	train's auc: 0.817254	valid's auc: 0.787079
[685]	train's auc: 0.817301	valid's auc: 0.787091
[686]

...

[1280]	train's auc: 0.841207	valid's auc: 0.790924
[1281]	train's auc: 0.841251	valid's auc: 0.790934
[1282]	train's auc: 0.841291	valid's auc: 0.790936
[1283]	train's auc: 0.841329	valid's auc: 0.790949
[1284]	train's auc: 0.841375	valid's auc: 0.790952
[1285]	train's auc: 0.841412	valid's auc: 0.790951
[1286]	train's auc: 0.841446	valid's auc: 0.790965
[1287]	train's auc: 0.841478	valid's auc: 0.790974
[1288]	train's auc: 0.84151	valid's auc: 0.790979
[1289]	train's auc: 0.841542	valid's auc: 0.79099
[1290]	train's auc: 0.841576	valid's auc: 0.790997
[1291]	train's auc: 0.841611	valid's auc: 0.790999
[1292]	train's auc: 0.841647	valid's auc: 0.790999
[1293]	train's auc: 0.841685	valid's auc: 0.791
[1294]	train's auc: 0.841721	valid's auc: 0.791007
[1295]	train's auc: 0.84176	valid's auc: 0.791009
[1296]	train's auc: 0.841794	valid's auc: 0.791023
[1297]	train's auc: 0.841824	valid's auc: 0.791025
[1298]	train's auc: 0.841859	valid's auc: 0.791032
[1299]	train's auc: 0.841896	valid's 

[1445]	train's auc: 0.846604	valid's auc: 0.791723
[1446]	train's auc: 0.846639	valid's auc: 0.791732
[1447]	train's auc: 0.84667	valid's auc: 0.791742
[1448]	train's auc: 0.846699	valid's auc: 0.791742
[1449]	train's auc: 0.846732	valid's auc: 0.79175
[1450]	train's auc: 0.846761	valid's auc: 0.791751
[1451]	train's auc: 0.846805	valid's auc: 0.791767
[1452]	train's auc: 0.846835	valid's auc: 0.791779
[1453]	train's auc: 0.846869	valid's auc: 0.791785
[1454]	train's auc: 0.846899	valid's auc: 0.791783
[1455]	train's auc: 0.846934	valid's auc: 0.791785
[1456]	train's auc: 0.84696	valid's auc: 0.791782
[1457]	train's auc: 0.846986	valid's auc: 0.791784
[1458]	train's auc: 0.847022	valid's auc: 0.791802
[1459]	train's auc: 0.847055	valid's auc: 0.7918
[1460]	train's auc: 0.847085	valid's auc: 0.791803
[1461]	train's auc: 0.847115	valid's auc: 0.791806
[1462]	train's auc: 0.84715	valid's auc: 0.791809
[1463]	train's auc: 0.847189	valid's auc: 0.791828
[1464]	train's auc: 0.847231	valid's 

[1607]	train's auc: 0.851646	valid's auc: 0.792289
[1608]	train's auc: 0.851666	valid's auc: 0.792295
[1609]	train's auc: 0.851692	valid's auc: 0.792298
[1610]	train's auc: 0.851725	valid's auc: 0.792306
[1611]	train's auc: 0.85175	valid's auc: 0.792298
[1612]	train's auc: 0.85178	valid's auc: 0.792305
[1613]	train's auc: 0.851808	valid's auc: 0.7923
[1614]	train's auc: 0.851836	valid's auc: 0.792304
[1615]	train's auc: 0.851862	valid's auc: 0.792307
[1616]	train's auc: 0.851885	valid's auc: 0.792306
[1617]	train's auc: 0.851923	valid's auc: 0.792319
[1618]	train's auc: 0.851955	valid's auc: 0.79233
[1619]	train's auc: 0.851986	valid's auc: 0.792335
[1620]	train's auc: 0.852016	valid's auc: 0.79233
[1621]	train's auc: 0.852057	valid's auc: 0.792326
[1622]	train's auc: 0.852085	valid's auc: 0.792328
[1623]	train's auc: 0.852115	valid's auc: 0.792329
[1624]	train's auc: 0.852153	valid's auc: 0.792328
[1625]	train's auc: 0.852176	valid's auc: 0.792328
[1626]	train's auc: 0.852211	valid's 

[1769]	train's auc: 0.856368	valid's auc: 0.792734
[1770]	train's auc: 0.856402	valid's auc: 0.792731
[1771]	train's auc: 0.856441	valid's auc: 0.792732
[1772]	train's auc: 0.856478	valid's auc: 0.792745
[1773]	train's auc: 0.856499	valid's auc: 0.792756
[1774]	train's auc: 0.856529	valid's auc: 0.792756
[1775]	train's auc: 0.856556	valid's auc: 0.792746
[1776]	train's auc: 0.856592	valid's auc: 0.792744
[1777]	train's auc: 0.856623	valid's auc: 0.79275
[1778]	train's auc: 0.856645	valid's auc: 0.792747
[1779]	train's auc: 0.856667	valid's auc: 0.792748
[1780]	train's auc: 0.856699	valid's auc: 0.792762
[1781]	train's auc: 0.856733	valid's auc: 0.79276
[1782]	train's auc: 0.856766	valid's auc: 0.792766
[1783]	train's auc: 0.856791	valid's auc: 0.792766
[1784]	train's auc: 0.856817	valid's auc: 0.792767
[1785]	train's auc: 0.856848	valid's auc: 0.792774
[1786]	train's auc: 0.856875	valid's auc: 0.792774
[1787]	train's auc: 0.856909	valid's auc: 0.792774
[1788]	train's auc: 0.856932	vali

[1932]	train's auc: 0.860828	valid's auc: 0.793015
[1933]	train's auc: 0.860854	valid's auc: 0.793012
[1934]	train's auc: 0.860874	valid's auc: 0.793021
[1935]	train's auc: 0.860902	valid's auc: 0.793021
[1936]	train's auc: 0.860923	valid's auc: 0.793019
[1937]	train's auc: 0.860946	valid's auc: 0.793017
[1938]	train's auc: 0.860983	valid's auc: 0.793015
[1939]	train's auc: 0.861	valid's auc: 0.793015
[1940]	train's auc: 0.861029	valid's auc: 0.793017
[1941]	train's auc: 0.861064	valid's auc: 0.793021
[1942]	train's auc: 0.861099	valid's auc: 0.793033
[1943]	train's auc: 0.861123	valid's auc: 0.79303
[1944]	train's auc: 0.861141	valid's auc: 0.793036
[1945]	train's auc: 0.861167	valid's auc: 0.793033
[1946]	train's auc: 0.861191	valid's auc: 0.793051
[1947]	train's auc: 0.861221	valid's auc: 0.793055
[1948]	train's auc: 0.861245	valid's auc: 0.793059
[1949]	train's auc: 0.861273	valid's auc: 0.793063
[1950]	train's auc: 0.861307	valid's auc: 0.793072
[1951]	train's auc: 0.861328	valid'

In [33]:
pred_test_1b

array([0.04276936, 0.10369423, 0.0360059 , ..., 0.01008144, 0.0285283 ,
       0.25686217])

In [34]:
pred_train_1b

array([0.24249863, 0.01785708, 0.03320322, ..., 0.03864351, 0.11520511,
       0.08183016])

In [35]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [36]:
print(type(pred_1b))

<class 'list'>


In [37]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2a': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set2a
0,100002,0.242499
1,100003,0.017857
2,100004,0.033203
3,100006,0.048900
4,100007,0.043556


In [38]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set2a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [39]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [40]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2a': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [41]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set2a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [42]:
del df
del x_train
del y_train
del x_test
del app_train
del app_test
del model_1b
gc.collect()

0

## 特徴量生成2